## 패키지 참조

In [ ]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 폰트세팅

In [ ]:
plt.rcParams["font.family"] = 'AppleGothic' if sys.platform == 'darwin' else 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.unicode_minus"] = False

## 데이터

In [ ]:
origin=read_excel('행정동병합최종(반포본동).xlsx')
origin.head(2)

### 분석에 필요한 변수만 추출

`읍면동`, `지수` 추출

In [ ]:
origin1=origin.filter(['읍면동명','환경자족성지수minmax','여가생활지수minmax','사회적약자_지원_인프라_밀집도minmax','접근성지수minmax'])
origin1.head(2)

### 이상치/결측치 확인 및 처리

In [ ]:
origin1.isna().sum()

In [ ]:
origin2=origin1.copy()
origin2=origin1.fillna(0)
origin2.isna().sum()

In [ ]:
origin2.head(2)

## #1. `환경자족성지수 - 여가생활지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin3=origin2.copy()
X = origin3.iloc[:,[1,2]].values # x값으로 환경자족성지수,여가생활지수 지정
X[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan=DBSCAN(eps=0.03,min_samples=5,leaf_size=10,metric='chebyshev')
dbscan.fit(X)

### 군집 결과

In [ ]:
cluster_label=dbscan.labels_
cluster_label

### 군집 결과 데이터 통합

In [ ]:
origin3['cluster_id'] = cluster_label
origin3.head()

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin3,x='여가생활지수minmax',y='환경자족성지수minmax',hue=cluster_label,palette='Set1')
plt.grid()
plt.show()
plt.close()

### 수동 voting을 위한 xlsx 파일 생성

In [ ]:
origin3[origin3['cluster_id']==-1]
origin3_final=DataFrame(origin3[origin3['cluster_id']==-1])
origin3_final.head()

In [ ]:
origin3_final.to_excel('./교집합결과/여가생활,환경자족성지수voting2.xlsx')

## #2. `환경자족성지수 - 사회적약자지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin4=origin2.copy()
Y = origin4.iloc[:,[1,3]].values # x값으로 환경자족성지수,사회적약자지수 지정
Y[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan1=DBSCAN(eps=0.03,min_samples=5,leaf_size=10,metric='chebyshev')
dbscan1.fit(Y)

### 군집 결과

In [ ]:
cluster_label1=dbscan1.labels_
cluster_label1

### 군집 결과 데이터 통합

In [ ]:
origin4['cluster_id'] = cluster_label1
origin4.head()

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin4,x='사회적약자_지원_인프라_밀집도minmax',y='환경자족성지수minmax',hue=cluster_label1,palette='Set1')
plt.grid()
plt.show()
plt.close()

### 수동 voting을 위한 xlsx 파일 생성

In [ ]:
origin4_final=DataFrame(origin4[origin4['cluster_id']==-1])
origin4_final.head()

In [ ]:
origin4_final.to_excel('./교집합결과/사회적약자,환경자족성지수voting2.xlsx')

## #.3 `환경자족성지수 - 접근성지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin5=origin2.copy()
Z = origin5.iloc[:,[1,4]].values # x값으로 환경자족성지수,접근성지수 지정
Z[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan2=DBSCAN(eps=0.05,min_samples=20,leaf_size=20,metric='chebyshev')
dbscan2.fit(Z)

### 군집 결과

In [ ]:
cluster_label2=dbscan2.labels_
cluster_label2

### 군집 결과 데이터 통합

In [ ]:
origin5['cluster_id'] = cluster_label2
origin5.head()

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin5,x='접근성지수minmax',y='환경자족성지수minmax',hue=cluster_label2,palette='Set1')
plt.grid()
plt.show()
plt.close()

### 수동 voting을 위한 xlsx 파일 생

In [ ]:
len(origin5[origin5['cluster_id']==-1])

In [ ]:
origin5_final=DataFrame(origin5[origin5['cluster_id']==-1])
origin5_final.head()

In [ ]:
origin5_final.to_excel('./교집합결과/접근성지수,환경자족성지수voting2.xlsx')

## #4. `여가생활지수 - 사회적약자지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin6=origin2.copy()
XX = origin6.iloc[:,[2,3]].values #x값으로 여가생활지수,사회적약자지수 지정
XX[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan3=DBSCAN(eps=0.04,min_samples=20,leaf_size=20,metric='chebyshev')
dbscan3.fit(XX)

### 군집 결과

In [ ]:
cluster_label3=dbscan3.labels_
cluster_label3

### 군집 결과 데이터 통합

In [ ]:
origin6['cluster_id'] = cluster_label3
origin6.head()

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin6,x='여가생활지수minmax',y='사회적약자_지원_인프라_밀집도minmax',hue=cluster_label3,palette='Set1')
plt.grid()
plt.show()
plt.close()

> 여기서는 이상치(`군집 -1`)를  선정한다. <br/> 분석의 척도가 될 지수가 긍정적인 장소를 추출해 시범 사업지로 선정할 계획.

### 수동 voting을 위한 xlsx 파일 생성

In [ ]:
origin6_final=DataFrame(origin6[origin6['cluster_id']==-1])
origin6_final.head()

In [ ]:
origin6_final.to_excel('./교집합결과/여가생활,사회적약자voting2.xlsx')

## #5. `여가생활지수 - 접근성지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin7=origin2.copy()
YY = origin7.iloc[:,[2,4]].values #x값으로 여가생활지수,접근성지수 지정
YY[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan4=DBSCAN(eps=0.03,min_samples=5,leaf_size=10,metric='chebyshev')
dbscan4.fit(YY)

### 군집 결과

In [ ]:
cluster_label4=dbscan4.labels_
cluster_label4

### 군집 결과 데이터 통합

In [ ]:
origin7['cluster_id'] = cluster_label4
origin7.head()

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin7,x='여가생활지수minmax',y='접근성지수minmax',hue=cluster_label4,palette='Set1')
plt.grid()
plt.show()
plt.close()

> 여기서는 이상치(`군집 -1`)를  선정한다. <br/> 비록 데이터가 산개되어 있지만 군집 `0`보다 각 지수 값이 보다 상위에 포진해있는 만큼 최적의 입지 후보군이 될 수 있다고 판단.

### 수동 voting을 위한 xlsx 파일 생성

In [ ]:
origin7_1final=DataFrame(origin7[origin7['cluster_id']==-1])
origin7_1final.head()

In [ ]:
origin7_1final.to_excel('./교집합결과/여가생활,접근성지수voting2.xlsx')

## #6. `사회적약자지수 - 접근성지수`

### 변수 추출

In [ ]:
origin2.head()

In [ ]:
origin8=origin2.copy()
ZZ = origin8.iloc[:,[3,4]].values #x값으로 사회적약자지수, 접근성지수 지정
ZZ[:5]

### DBSCAN 모델 구축

DBSCAN 구현 파라미터

| 파라미터 | 설명 |
| -- | -- |
| `eps` | 하나의 샘플에 대한 다른 샘플과의 최대 거리 |
| `min_samples` | MinPts. 핵심 포인트로 간주할 포인트에 대한 이웃의 샘플 수 |

In [ ]:
dbscan5=DBSCAN(eps=0.04,min_samples=10,leaf_size=10,metric='chebyshev')
dbscan5.fit(ZZ)

### 군집 결과

In [ ]:
cluster_label5=dbscan5.labels_
cluster_label5

### 군집 결과 데이터 통합

In [ ]:
origin8['cluster_id'] = cluster_label5
origin8

### 군집 결과 시각화

In [ ]:
plt.figure(figsize=(10,5))
sb.scatterplot(data=origin8,x='사회적약자_지원_인프라_밀집도minmaxminmax',y='접근성지수minmax',hue=cluster_label5,palette='Set1')
plt.grid()
plt.show()
plt.close()

> 여기서는 이상치(`군집 -1`)를  선정한다. <br/> 비록 데이터가 산개되어 있지만 군집 `0`보다 각 지수 값이 보다 상위에 포진해있는 만큼 최적의 입지 후보군이 될 수 있다고 판단.

### 수동 voting을 위한 xlsx 파일 생성

In [ ]:
origin8_final=DataFrame(origin8[origin8['cluster_id']==-1])
origin8_final.head()

In [ ]:
origin8_final.to_excel('./교집합결과/사회적약자,접근성지수voting2.xlsx')

> - DBSCAN의 경우 이상치에 민감하다.
> - 파라미터를 전부 같은 값으로 설정하고 군집모델을 실행했을 때 <br/> 일부는 적절하게 분리가 됐지만 일부는 한 개의 군집만 판별하고 나머지는 전부 이상치로 판단하였다.
> - 이상치라 하더라도 데이터 분석 결과 군집이 특정 지역에 몰려있거나 두 지수 모두 하위권에 분포할 경우, `두 지수중 하나라도 상위권에 분포한 이상치를 후보군으로 선정` 한다. <br/> ==> 적절한 데이터(군집)으로 판단한다.
>    - 이유는 `어떤 한 지수라도 높아야 보행일상권 대상에 포함이 되기 때문`
> - 결론적으로 KMeans로 추출한 `행정동과 DBSCAN으로 추출한 지수가 높은 행정동을 비교`해 `지수가 높은 행정동을 중심으로 최적의 입지 후보지라고 판단`.